In [1]:
import sympy as sym

# Kalman Filter Primer

Components of a Kalman filter:
- Matrix $A_t$ is a linear transformation that maps a previous state vector $x_{t-1}$ to the current state $x_t$, $A_t \in \mathbb{R}^{n \times n}$.
- Matrix $B_t$ is a linear transformation that maps a control vector $u_t$ to the current state $x_t$, $B_t \in \mathbb{R}^{n \times l}$.
- where $n$ is the dimension of the state vector and $l$ is the dimension of the control vector.
- Matrix $C_t$ is a linear transformation that maps the state vector $x_t$ to the observation vector $z_t$, $C_t \in \mathbb{R}^{k \times n}$.

we can write the state transition function as
$$
x_t = A_t x_{t-1} + B_t u_t + \epsilon_t
$$
where $\epsilon_t$ is the process noise, it is normally distributed with covariance $R_t$. The observation function is written as
$$
z_t = C_t x_t + \delta_t
$$
where $\delta_t$ is the observation noise, also normally distributed with covariance $Q_t$. 

The Kalman filter is a recursive algorithm that estimates the state of a linear dynamic system from a series of noisy observations. The Kalman filter is a linear estimator that minimizes the mean squared error of the estimate. The Kalman filter is a two-step process: prediction and update. In the prediction step, the filter estimates the state of the system at the next time step. In the update step, the filter updates the state estimate based on the new observation. The Kalman filter is optimal in the sense that it minimizes the mean squared error of the estimate.


The Kalman Filter algorithm is as follows:

0. Initialize the state estimate $x_0$ and the error covariance $\Sigma_0$ for $t = 0$.
1. $\hat{x}_t = A_t x_{t-1} + B_t u_t$
2. $\hat{\Sigma_t} = A_t \Sigma_{t-1} A_t^T + R_t$
3. $K_t = \hat{\Sigma_t} C_t^T (C_t \hat{\Sigma_t} C_t^T + Q_t)^{-1}$
4. $x_t = \hat{x}_t + K_t (z_t - C_t \hat{x}_t)$
5. $\Sigma_t = (I - K_t C_t) \hat{\Sigma_t}$



# Extended Kalman Filter Derivations

The Extended Kalman Filter (EKF) is an extension of the Kalman filter that can handle non-linear systems. The EKF linearizes the system dynamics and observation functions around the current state estimate. The EKF uses the linearized models to perform the prediction and update steps of the Kalman filter algorithm. The EKF is an approximation of the true non-linear system, and the quality of the estimate depends on the accuracy of the linearization.

## EKF Linearisation: First Order Taylor Expansion
 - Prediction:
$$
\begin{align}
g(u_t, x_{t-1}) \approx& g(u_t, \hat{x}_{t-1}) + G_t (x_{t-1} - \hat{x}_{t-1})\\
G_t =& \frac{\partial g(u_t, \hat{x}_{t-1})}{\partial x_{t-1}}
\end{align}
$$
 - Correction:
$$
\begin{align}
h(x_t) \approx& h(\hat{x}_t) + H_t (x_t - \hat{x}_t)\\
H_t =& \frac{\partial h(\hat{x}_t)}{\partial x_t}
\end{align}
$$
where $G_t$ and $H_t$ are the Jacobians of the system dynamics and observation functions, respectively.

## EKF Algorithm
0. Initialize the state estimate $x_0$ and the error covariance $\Sigma_0$.
1. $\hat{x}_t = g(u_t, \hat{x}_{t-1})$
2. $\hat{\Sigma_t} = G_t \Sigma_{t-1} G_t^T + R_t$
3. $K_t = \hat{\Sigma_t} H_t^T (H_t \hat{\Sigma_t} H_t^T + Q_t)^{-1}$
4. $x_t = \hat{x}_t + K_t (z_t - h(\hat{x}_t))$
5. $\Sigma_t = (I - K_t H_t) \hat{\Sigma_t}$

# EKF for SLAM
1. State prediction
2. Measurement prediction
3. Measurement
4. Data association
5. Update

## Applied to real-world problems

With a given motion model of 
$$
\begin{align}
x_t &= x_{t-1} + \delta_{trans} \cos(\theta_{t-1} + \delta_{rot1}) \\
y_t &= y_{t-1} + \delta_{trans} \sin(\theta_{t-1} + \delta_{rot1}) \\
\theta_t &= \theta_{t-1} + \delta_{rot1} + \delta_{rot2}
\end{align}
$$

In [2]:
# Define the symbols
x, y, theta, d_trans, d_rot1, d_rot2 = sym.symbols('x y theta d_trans d_rot1 d_rot2')
f = sym.Matrix([
    x + d_trans*sym.cos(theta + d_rot1),
    y + d_trans*sym.sin(theta + d_rot1),
    theta + d_rot1 + d_rot2])

In [3]:
f

Matrix([
[d_trans*cos(d_rot1 + theta) + x],
[d_trans*sin(d_rot1 + theta) + y],
[        d_rot1 + d_rot2 + theta]])

In [4]:
G = f.jacobian([x, y, theta])

In [5]:
G

Matrix([
[1, 0, -d_trans*sin(d_rot1 + theta)],
[0, 1,  d_trans*cos(d_rot1 + theta)],
[0, 0,                            1]])

Given a robot position and heading 
$$
\mu = \begin{bmatrix}
\mu_{t, x} \\
\mu_{t, y} \\
\mu_{t, \theta}
\end{bmatrix}
$$
we define the homogeneous transformation matrix as
$$
\begin{align}
T &= \begin{bmatrix}
\cos(\mu_{t, \theta}) & -\sin(\mu_{t, \theta}) & \mu_{t, x} \\
\sin(\mu_{t, \theta}) & \cos(\mu_{t, \theta}) & \mu_{t, y} \\
0 & 0 & 1
\end{bmatrix}
\end{align}
$$
and a sensor observation as 
$$
\begin{align}
z_t &= \begin{bmatrix}
r_t * \cos(\phi_t) \\
r_t * \sin(\phi_t) \\
0
\end{bmatrix}
\end{align}
$$
we obtain the relative end point as

In [6]:
mu_x, mu_y, mu_theta, r, phi = sym.symbols('mu_x mu_y mu_theta r phi')
z = sym.Matrix([
    r*sym.cos(phi),
    r*sym.sin(phi),
    0])
T_mu = sym.Matrix([
    [sym.cos(mu_theta), -sym.sin(mu_theta), mu_x],
    [sym.sin(mu_theta), sym.cos(mu_theta), mu_y],
    [0, 0, 1]])
T_z = sym.Matrix([
    [sym.cos(0), -sym.sin(0), r*sym.cos(phi)],
    [sym.sin(0), sym.cos(0), r*sym.sin(phi)],
    [0, 0, 1]])
T_final = T_mu @ T_z
T_final

Matrix([
[cos(mu_theta), -sin(mu_theta), mu_x - r*sin(mu_theta)*sin(phi) + r*cos(mu_theta)*cos(phi)],
[sin(mu_theta),  cos(mu_theta), mu_y + r*sin(mu_theta)*cos(phi) + r*sin(phi)*cos(mu_theta)],
[            0,              0,                                                          1]])

In [7]:
T_vec = T_final[:2, 2]
T_vec.applyfunc(sym.simplify)

Matrix([
[mu_x + r*cos(mu_theta + phi)],
[mu_y + r*sin(mu_theta + phi)]])

The observation model is given by (for a range bearing sensor):
for an observation $z_{j,t}^i = (r_{t}^i, \phi_{t}^i)$ of a landmark $i$ at time $t$:
$$
\begin{align}
\hat{\mu}_{j,x} &= \hat{\mu}_{t, x} + r_{t}^i \cos(\phi_{t}^i + \hat{\mu}_{t, \theta} ) \\
\hat{\mu}_{j,y} &= \hat{\mu}_{t, y} + r_{t}^i \sin(\phi_{t}^i + \hat{\mu}_{t, \theta}) \\
\end{align}
$$
where $\hat{\mu}_{j,x}$ is the observed location of landmark $j$, $\hat{\mu}_{t, x}$ is the estimated robot location at time $t$, and the rest is the relative measurement of the landmark $i$ to the robot at time $t$.

The expectation of the observation model is then given by:
$$
\begin{align}
\delta &= 
\begin{bmatrix}
\delta_x\\
\delta_y
\end{bmatrix} = \begin{bmatrix}
\hat{\mu}_{j,x} - \hat{\mu}_{t, x} \\
\hat{\mu}_{j,y} - \hat{\mu}_{t, y}
\end{bmatrix} \\
q &= \delta^T\delta \\
\hat{z}_t^i &= \begin{bmatrix}
\sqrt{q} \\
\text{atan2}(\delta_y, \delta_x) - \mu_{t,\theta}
\end{bmatrix}
\end{align}
$$

The Correction step consists of the following:
1. Associate the landmarks with the measurements (or assume known associations for the example)
2. Initialise landmarks if unobserved
3. Compute the expected observation
4. Compute the Jacobian of the observation model
5. Compute the Kalman gain

In [8]:
# Define the symbols
mu_x, mu_y, mu_theta, m_x, m_y = sym.symbols('mu_x mu_y mu_theta m_x m_y')

In [9]:
delta = sym.Matrix([
    m_x - mu_x,
    m_y - mu_y])
h = sym.Matrix([
    sym.sqrt(delta[0]**2 + delta[1]**2),
    sym.atan2(delta[1], delta[0]) - mu_theta])


In [10]:
h

Matrix([
[  sqrt((m_x - mu_x)**2 + (m_y - mu_y)**2)],
[-mu_theta + atan2(m_y - mu_y, m_x - mu_x)]])

In [11]:
H = h.jacobian([mu_x, mu_y, mu_theta, m_x, m_y])

In [12]:
H

Matrix([
[(-m_x + mu_x)/sqrt((m_x - mu_x)**2 + (m_y - mu_y)**2), (-m_y + mu_y)/sqrt((m_x - mu_x)**2 + (m_y - mu_y)**2),  0, (m_x - mu_x)/sqrt((m_x - mu_x)**2 + (m_y - mu_y)**2), (m_y - mu_y)/sqrt((m_x - mu_x)**2 + (m_y - mu_y)**2)],
[   -(-m_y + mu_y)/((m_x - mu_x)**2 + (m_y - mu_y)**2),     -(m_x - mu_x)/((m_x - mu_x)**2 + (m_y - mu_y)**2), -1,    (-m_y + mu_y)/((m_x - mu_x)**2 + (m_y - mu_y)**2),     (m_x - mu_x)/((m_x - mu_x)**2 + (m_y - mu_y)**2)]])

which we can simplify the notation to:
$$
\begin{align}
H_t &= \frac{1}{q}\begin{bmatrix}
-\sqrt{q}\delta_x & -\sqrt{q}\delta_y & 0 & \sqrt{q}\delta_x & \sqrt{q}\delta_y \\
\delta_y & -\delta_x & -q & -\delta_y & \delta_x
\end{bmatrix}
\end{align}
$$